In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf as pdf
import matplotlib.patches as pch

import scipy.stats
import eleanor_constants as EL

matplotlib.rc('text', usetex=True)
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
matplotlib.rcParams['svg.fonttype'] = 'path'
matplotlib.rcParams['axes.titlepad'] = 19
%matplotlib inline

savename = "./figures/S3.pdf"

In [ ]:
df = pd.read_csv('./data/trajectories/summary/cleaned_animal_analyses.csv')
df["discovery_time_diff"] = df["discovery_time_diff"]/60 # convert to minutes for cleaner plots
labels = ["F", "FE", "Y", "Q", "I", "O", "W"]

fed_data = []
temp1 = df[df['A_starved'] == 'no']
for label in labels:
    treat = EL.treatments.get(label)
    temp = temp1[temp1['A_treatment_odor'] == treat]
    fed_data.append(temp)
    
str_data = []
temp1 = df[df['A_starved'] == '1day']
for label in labels:
    treat = EL.treatments.get(label)
    temp = temp1[temp1['A_treatment_odor'] == treat]
    str_data.append(temp)
    
lens = [len(df) for df in fed_data]
print(lens)
lens = [len(df) for df in str_data]
print(lens)

labels = ["Food", "Food extract", "Yeast RNA", "Quinine", "Indole", "O-cresol", "Water"]
colors = [EL.c_food, EL.c_extract, EL.c_rna, EL.c_quinine, EL.c_indole, EL.c_cresol, EL.c_water]
colors = colors + ["k"] + colors

In [ ]:
fig = plt.figure(figsize=(14, 10))

# SET UP FIGURES -----------------------------------------------------------------
ax1 = fig.add_subplot(3, 2, 1, facecolor="None")
ax2 = fig.add_subplot(3, 2, 2, facecolor="None")
ax3 = fig.add_subplot(3, 2, 3, facecolor="None")
ax4 = fig.add_subplot(3, 2, 4, facecolor="None")
ax5 = fig.add_subplot(3, 2, 5, facecolor="None")

v = 1.2
ax1.set_ylim(-v, v)
ax1.set_yticks(np.arange(-v, v+0.1, 0.2), minor=True)
ax1.set_yticks([-v, 0, v])

v = 15
ax2.set_ylim(-v, v)
ax2.set_yticks(np.arange(-v, v+0.1, 3), minor=True)
ax2.set_yticks([-v, 0, v])

v = 0.8
ax3.set_ylim(-v, v)
ax3.set_yticks(np.arange(-v, v+0.1, 0.2), minor=True)
ax3.set_yticks([-v, 0, v])

v = 0.8
ax4.set_ylim(-v, v)
ax4.set_yticks(np.arange(-v, v+0.1, 0.2), minor=True)
ax4.set_yticks([-v, 0, v])

v = 1.4
ax5.set_ylim(-v, v)
ax5.set_yticks(np.arange(-v, v+0.1, 0.2), minor=True)
ax5.set_yticks([-v, 0, v])

labels = ["Food", "Food extract", "Yeast RNA", "Quinine", "Indole", "O-cresol", "Water"]

# PLOT DATA ------------------------------------------------------------------------
axes = [ax1, ax2, ax3, ax4, ax5]
tests = ["cd_move_diff", "discovery_time_diff", "c_turn_diff", "cd_turn_diff", "cd_speed_diff"]
eqs = [r"${DP = \frac{T_{up} - T_{down}} {T_{up} + T_{down}} \quad \Delta DP = DP_{Experiment}-DP_{Acclimation}}$",
       r"${D = T_{high[1]} \quad \Delta D = D_{Experiment}-D_{Acclimation}}$",
    r"${CTI = \frac{Turns_{high}}{T_{high}} - \frac{Turns_{low}}{T_{low}}\quad \Delta CTI = CTI_{Experiment}-CTI_{Acclimation}}$",
    r"${DTI = \frac{Turns_{up}}{T_{up}} - \frac{Turns_{down}}{T_{down}}\quad \Delta DTI = DTI_{Experiment}-DTI_{Acclimation}}$",
    r"${DS = \tilde{S}_{up} - \tilde{S}_{down} \quad \Delta DS = DS_{Experiment}-DS_{Acclimation}}$"
      ]
ys = [r"${\Delta DP}$", r"${\Delta D}$", r"${\Delta CTI}$", r"${\Delta DTI}$", r"${\Delta DS}$"]
  
for n, (test, ax) in enumerate(zip(tests, [ax1, ax2, ax3, ax4, ax5])):
    str_datum = [df[test].values for df in str_data]
    str_datum = [[x for x in v if ~np.isnan(x)] for v in str_datum]
    fed_datum = [df[test].values for df in fed_data]
    fed_datum = [[x for x in v if ~np.isnan(x)] for v in fed_datum]
    
    datamax = max([max(v+z) for v, z in zip(str_datum, fed_datum)])
    datamin = min([min(v+z) for v, z in zip(str_datum, fed_datum)])
    print(test, "values range from", round(datamax, 2), round(datamin, 2))
    
    datum = str_datum + [[0]] + fed_datum
    
    jitter = [np.random.normal(scale=0.1, size=len(x)) for x in datum]
    parts = ax.violinplot(datum, showmeans=False, showmedians=False, showextrema=False)
    box = ax.boxplot(datum, showfliers=False)
    
    for i, pc in enumerate(parts['bodies']):
        pc.set_facecolor(colors[i])
        pc.set_alpha(0.25)
        
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(box[element], color="k", alpha=0.35, lw=1.25, clip_on=False, zorder=100)

    for i, (j, d) in enumerate(zip(jitter, datum)):
        j = [x+i+1 for x in j]
        if len(j) > 1:
            ax.scatter(j, d, alpha=0.5, s=EL.s, color=colors[i], clip_on=False, zorder=20)
            ax.scatter([i+1], np.mean(d), color="k", s=EL.s+30, alpha=0.5, zorder=21, marker="+", clip_on=False)
            
    gap = 0.02
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.xaxis.set_ticks_position('none') 
    ax.set_xticks(np.arange(1, 16, 1))
    ax.set_xlim(0.25, 15.75)
    ax.set_xticklabels([])
    
    # Horizontal line at 0 for reference
    ax.axhline(y=0, xmin=0+gap, xmax=0.5-(gap/1.5), color="k", alpha=0.5, lw=1, zorder=200, clip_on=False)
    ax.axhline(y=0, xmin=0.5+(gap/1.5), xmax=1-gap, color="k", alpha=0.5, lw=1, zorder=200, clip_on=False)

    ax.set_ylabel(ys[n])
    fsize = 16
    if n > 1:
        fsize = 14
    ax.set_title(eqs[n], fontsize=fsize, alpha=0.75)
    
    # White box to cover up spacer boxplot
    low, high = ax.get_ylim()
    bit = (high-low)*0.05
    ax.add_patch(pch.Rectangle((7.5, low-bit), 1, (high-low)+bit, 
                               facecolor="#ffffff", edgecolor="none", zorder=110, clip_on=False))
    if (ax == ax4) or (ax == ax5):
        ax.text(4, low+bit*1.75, "Starved animals", clip_on=False, ha="center", va="top", zorder=12)
        ax.text(12, low+bit*1.75, "Fed animals", clip_on=False, ha="center", va="top", zorder=12)
        ax.set_xticklabels(labels + [" "] + labels, rotation=90)
        ax.axhline(y=low, xmin=0+gap, xmax=0.5-(gap/1.5), color="k", alpha=0.5, lw=1, clip_on=False)
        ax.axhline(y=low, xmin=0.5+(gap/1.5), xmax=1-gap, color="k", alpha=0.5, lw=1, clip_on=False)
        ax.xaxis.set_ticks_position('bottom') 
            
# SET BOUNDARIES AND SAVE FIGURE -----------------------------------------------
plt.tight_layout()
fig.subplots_adjust(wspace=0.2)
pp = pdf.PdfPages(savename, keep_empty=False)
pp.savefig(fig)
pp.close()
plt.show()